In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read files
confirmed_data = pd.read_csv('time_series_covid19_confirmed_US.csv')
death_data = pd.read_csv('time_series_covid19_deaths_US.csv')
county_data = pd.read_csv('abridged_couties.csv')


In [3]:
confirmed_data.head()
#The "Admin2" column refers to the county name, rename the column
confirmed_data = confirmed_data.rename(columns={'Admin2':'County'})

In [4]:
death_data.head()
#The "Admin2" column refers to the county name, rename the column
death_data = death_data.rename(columns={'Admin2':'County'})

In [5]:
county_data.head()

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,POP_LONGITUDE,...,>500 gatherings,public schools,restaurant dine-in,entertainment/gym,federal guidelines,foreign travel ban,SVIPercentile,HPSAShortage,HPSAServedPop,HPSAUnderservedPop
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,-86.494165,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4354,NaN,NaN,NaN
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,-87.762381,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.2162,NaN,NaN,NaN
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,-85.310038,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.9959,6.08,5400.0,18241.0
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,-87.127659,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.6003,2.75,14980.0,6120.0
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,-86.591491,...,737497.0,737500.0,737503.0,737512.0,737500.0,737495.0,0.4242,7.21,31850.0,25233.0


In [6]:
#Since we will work on this data on a county basis, first check if there are any null values for county names and state names in each dataset
print("Number of null values in confirmed data for county names is ",confirmed_data['County'].isnull().sum())
print("Number of null values in death data for county names is ",death_data['County'].isnull().sum())
print("Number of null values in county feature data for county names is ",county_data['CountyName'].isnull().sum())

print("Number of null values in confirmed data for state names is ",confirmed_data['Province_State'].isnull().sum())
print("Number of null values in death data for state names is ",death_data['Province_State'].isnull().sum())
print("Number of null values in county feature data for state names is ",county_data['State'].isnull().sum())

Number of null values in confirmed data for county names is  7
Number of null values in death data for county names is  7
Number of null values in county feature data for county names is  0
Number of null values in confirmed data for state names is  0
Number of null values in death data for state names is  0
Number of null values in county feature data for state names is  169


In [7]:
#Drop the NaN county names for the confirmed and death time series dataset
confirmed_data = confirmed_data[confirmed_data['County'].notna()]
county_data = county_data[county_data['CountyName'].notna()]
death_data = death_data[death_data['County'].notna()]

# First look at the county level feature dataset

#  There appear to be alot of null values in the state names in the county feature dataset, but many of these have the abbreviated state names filled in without the full state name, so let's fill them in

In [8]:
county_data.loc[(county_data.StateName=="FL"),"State"]="Florida"
county_data.loc[(county_data.StateName=="HI"),"State"]="Hawaii"
county_data.loc[(county_data.StateName=="MT"),"State"]="Montana"
county_data.loc[(county_data.StateName=="VA"),"State"]="Virginia"
county_data.loc[(county_data.StateName=="NY"),"State"]="New York"

In [9]:
#See which states are included in the county level dataset
county_data['State'].value_counts()

Texas                   254
Georgia                 159
Virginia                136
Kentucky                120
Missouri                115
Kansas                  105
Illinois                102
North Carolina          100
Iowa                     99
Tennessee                95
Nebraska                 93
Indiana                  92
Ohio                     88
Minnesota                87
Michigan                 83
Mississippi              82
Oklahoma                 77
Arkansas                 75
Wisconsin                72
Florida                  68
Pennsylvania             67
Alabama                  67
South Dakota             66
Louisiana                64
Colorado                 64
New York                 63
California               58
Montana                  57
West Virginia            55
North Dakota             53
South Carolina           46
Idaho                    44
Washington               39
Oregon                   36
New Mexico               33
Utah                

In [10]:
###Classify states as democratic or republican based on 2016 election results at the state level###
county_data['Political_Party'] = np.where(
    (county_data['State'] == "Washington") | (county_data['State'] == "Oregon") | (county_data['State'] == "Nevada") |
    (county_data['State'] == "California") | (county_data['State'] == "Colorado") | (county_data['State'] == "New Mexico") |
    (county_data['State'] == "Minnesota") | (county_data['State'] == "Illinois") |
    (county_data['State'] == "Vermont") | (county_data['State'] == "New Hampshire") | (county_data['State'] == "Maine") |
    (county_data['State'] == "Massachusetts") | (county_data['State'] == "Rhode Island") | (county_data['State'] == "Connecticut") |
    (county_data['State'] == "New Jersey") | (county_data['State'] == "Delaware") | (county_data['State'] == "Maryland") |
    (county_data['State'] == "District Of Columbia") | (county_data['State'] == "Virginia") | (county_data['State'] == "New York")
    , "Democrat","Republican")



In [10]:
###Classify democrat or republican based at county level###
county_data['Political_Party'] = np.where(county_data['dem_to_rep_ratio']>0.5,"Democrat","Republican")

In [11]:
#Create combined key which includes county name, state, and country which will be used for merging later
county_data['Combined_Key'] = county_data['CountyName']+','+' '+county_data['State']+','+' '+'US'

# Now look at the deaths time series dataset

In [12]:
#Only consider lower contiguous USA
death_data_us = death_data.query('Country_Region == "US"')

In [13]:
#See which states are in the dataset
death_data_us['Province_State'].value_counts()

Texas                   256
Georgia                 161
Virginia                135
Kentucky                122
Missouri                118
Kansas                  107
Illinois                104
North Carolina          102
Iowa                    101
Tennessee                97
Nebraska                 95
Indiana                  94
Ohio                     90
Minnesota                89
Michigan                 87
Mississippi              84
Oklahoma                 79
Arkansas                 77
Wisconsin                74
Pennsylvania             69
Alabama                  69
Florida                  69
South Dakota             68
Louisiana                66
Colorado                 66
New York                 64
California               60
Montana                  58
West Virginia            57
North Dakota             55
South Carolina           48
Idaho                    46
Washington               41
Oregon                   38
New Mexico               35
Utah                

In [14]:
#Since we are only looking at lower contiguous USA, delete other regions
death_data_us = death_data_us.query('Province_State != "Alaska"')
death_data_us = death_data_us.query('Province_State != "Hawaii"')
death_data_us = death_data_us.query('Province_State != "Guam"')
death_data_us = death_data_us.query('Province_State != "American Samoa"')
death_data_us = death_data_us.query('Province_State != "Puerto Rico"')
death_data_us = death_data_us.query('Province_State != "Diamond Princess"')
death_data_us = death_data_us.query('Province_State != "Grand Princess"')
death_data_us = death_data_us.query('Province_State != "Northern Mariana Islands"')
death_data_us = death_data_us.query('Province_State != "Virgin Islands"')

In [15]:
#See which counties are in the dataset
death_data_us['County'].value_counts()

Unassigned    49
Washington    31
Jefferson     26
Franklin      25
Jackson       24
              ..
Cortland       1
Yolo           1
Trigg          1
Troup          1
Laclede        1
Name: County, Length: 1861, dtype: int64

In [16]:
#Some counties appear to have values such as unassigned and "out of", we will delete these
death_data_us = death_data_us[~death_data_us.County.str.contains("Unassigned")]
death_data_us = death_data_us[~death_data_us.County.str.contains("Out of")]
#Set combined key as index which will be reset later for merging
death_data_us.set_index('Combined_Key',inplace=True)
#Consider death data from beginning day to end day, i.e Jan 22/2020 to April 18/2020 (Total 88 days)
death_data_us = death_data_us.iloc[:,-88:]

# Now look at the confirmed cases dataset

In [17]:
#Consider only lower contiguous USA
confirmed_data_us = confirmed_data.query('Country_Region == "US"')

In [18]:
#See which states are in the dataset
confirmed_data_us['Province_State'].value_counts()

Texas                   256
Georgia                 161
Virginia                135
Kentucky                122
Missouri                118
Kansas                  107
Illinois                104
North Carolina          102
Iowa                    101
Tennessee                97
Nebraska                 95
Indiana                  94
Ohio                     90
Minnesota                89
Michigan                 87
Mississippi              84
Oklahoma                 79
Arkansas                 77
Wisconsin                74
Pennsylvania             69
Alabama                  69
Florida                  69
South Dakota             68
Louisiana                66
Colorado                 66
New York                 64
California               60
Montana                  58
West Virginia            57
North Dakota             55
South Carolina           48
Idaho                    46
Washington               41
Oregon                   38
New Mexico               35
Utah                

In [19]:
#Since we are only looking at lower contiguous USA, delete other regions
confirmed_data_us = confirmed_data_us.query('Province_State != "Alaska"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Hawaii"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Guam"')
confirmed_data_us = confirmed_data_us.query('Province_State != "American Samoa"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Puerto Rico"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Diamond Princess"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Grand Princess"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Northern Mariana Islands"')
confirmed_data_us = confirmed_data_us.query('Province_State != "Virgin Islands"')

In [20]:
#See which counties are in the dataset
confirmed_data_us['County'].value_counts()

Unassigned    49
Washington    31
Jefferson     26
Franklin      25
Jackson       24
              ..
Cortland       1
Yolo           1
Trigg          1
Troup          1
Laclede        1
Name: County, Length: 1861, dtype: int64

In [21]:
#Some counties appear to have values such as unassigned and "out of", we will delete these
confirmed_data_us = confirmed_data_us[~confirmed_data_us.County.str.contains("Unassigned")]
confirmed_data_us = confirmed_data_us[~confirmed_data_us.County.str.contains("Out of")]
#Set combined key as index, which will be reset later for merging
confirmed_data_us.set_index('Combined_Key',inplace=True)
#Consider death data from beginning day to end day, i.e Jan 22/2020 to April 18/2020 (Total of 88 days)
confirmed_data_us = confirmed_data_us.iloc[:,-88:]

# Calculate average day to day growth rate for confirmed cases and deaths

In [22]:
total_dtd_confirmed = np.zeros(np.size(confirmed_data_us,0))
for i in range(np.size(confirmed_data_us,0)-1):
    for j in range(np.size(confirmed_data_us,1)-2):
        dtd_confirmed = confirmed_data_us.iloc[i,j+1]/confirmed_data_us.iloc[i,j]
        if np.isnan(dtd_confirmed) or np.isinf(dtd_confirmed):
            dtd_confirmed = 0
        total_dtd_confirmed[i] = total_dtd_confirmed[i] + dtd_confirmed

total_dtd_death = np.zeros(np.size(death_data_us,0))
for i in range(np.size(death_data_us,0)-1):
    for j in range(np.size(death_data_us,1)-2):
        dtd_death = death_data_us.iloc[i,j+1]/death_data_us.iloc[i,j]
        if np.isnan(dtd_death) or np.isinf(dtd_death):
            dtd_death = 0
        total_dtd_death[i] = total_dtd_death[i] + dtd_death

avg_dtd_confirmed = total_dtd_confirmed/np.size(confirmed_data_us,1)
confirmed_data_us['average_dtd_growth'] = avg_dtd_confirmed

avg_dtd_death = total_dtd_death/np.size(death_data_us,1)
death_data_us['average_dtd_death'] = avg_dtd_death

#Put both average confirmed cases and average deaths into the confirmed data dataframe for further manipulation
confirmed_data_us['avg_death_rate'] = death_data_us['average_dtd_death']
confirmed_data_us = confirmed_data_us.reset_index()

C:\Users\Jerry\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.
C:\Users\Jerry\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in longlong_scalars
  after removing the cwd from sys.path.
C:\Users\Jerry\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  if sys.path[0] == '':
C:\Users\Jerry\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in longlong_scalars
  if sys.path[0] == '':


In [23]:
confirmed_data_us.head()

,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,average_dtd_growth,avg_death_rate
0,"Autauga, Alabama, US",0,0,0,0,0,0,0,0,0,...,19,19,19,23,24,26,26,25,0.330657,0.125000
1,"Baldwin, Alabama, US",0,0,0,0,0,0,0,0,0,...,66,71,72,87,91,101,103,109,0.439499,0.227273
2,"Barbour, Alabama, US",0,0,0,0,0,0,0,0,0,...,9,10,10,11,12,14,15,18,0.200266,0.000000
3,"Bibb, Alabama, US",0,0,0,0,0,0,0,0,0,...,13,16,17,17,18,22,24,26,0.236596,0.000000
4,"Blount, Alabama, US",0,0,0,0,0,0,0,0,0,...,12,13,14,16,17,18,20,20,0.304404,0.000000


# Feature Engineering

In [44]:
#First, let's merge the calculated average growth/death rate data with the features of each county
joined_data = pd.merge(county_data,confirmed_data_us,on='Combined_Key',how='inner')

In [45]:
joined_data.head()

,countyFIPS,STATEFP,COUNTYFP,CountyName,StateName,State,lat,lon,POP_LATITUDE,POP_LONGITUDE,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,average_dtd_growth,avg_death_rate
0,01001,1.0,1.0,Autauga,AL,Alabama,32.540091,-86.645649,32.500389,-86.494165,...,19,19,19,23,24,26,26,25,0.330657,0.125000
1,01003,1.0,3.0,Baldwin,AL,Alabama,30.738314,-87.726272,30.548923,-87.762381,...,66,71,72,87,91,101,103,109,0.439499,0.227273
2,01005,1.0,5.0,Barbour,AL,Alabama,31.874030,-85.397327,31.844036,-85.310038,...,9,10,10,11,12,14,15,18,0.200266,0.000000
3,01007,1.0,7.0,Bibb,AL,Alabama,32.999024,-87.125260,33.030921,-87.127659,...,13,16,17,17,18,22,24,26,0.236596,0.000000
4,01009,1.0,9.0,Blount,AL,Alabama,33.990440,-86.562711,33.955243,-86.591491,...,12,13,14,16,17,18,20,20,0.304404,0.000000


In [46]:
#Drop some columns which contain information which we will not use
joined_data.drop(columns=['countyFIPS','STATEFP','COUNTYFP','POP_LATITUDE','POP_LONGITUDE',
                          'CensusRegionName','CensusDivisionName','Rural-UrbanContinuumCode2013'],inplace=True)

In [47]:
joined_data.set_index('Political_Party',inplace=True)
joined_data.reset_index(inplace=True)
joined_data.set_index('Combined_Key',inplace=True)
joined_data.reset_index(inplace=True)
joined_data

,Combined_Key,Political_Party,CountyName,StateName,State,lat,lon,PopulationEstimate2018,PopTotalMale2017,PopTotalFemale2017,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,average_dtd_growth,avg_death_rate
0,"Autauga, Alabama, US",Republican,Autauga,AL,Alabama,32.540091,-86.645649,55601.0,27007.0,28497.0,...,19,19,19,23,24,26,26,25,0.330657,0.125000
1,"Baldwin, Alabama, US",Republican,Baldwin,AL,Alabama,30.738314,-87.726272,218022.0,103225.0,109403.0,...,66,71,72,87,91,101,103,109,0.439499,0.227273
2,"Barbour, Alabama, US",Democrat,Barbour,AL,Alabama,31.874030,-85.397327,24881.0,13335.0,11935.0,...,9,10,10,11,12,14,15,18,0.200266,0.000000
3,"Bibb, Alabama, US",Republican,Bibb,AL,Alabama,32.999024,-87.125260,22400.0,12138.0,10530.0,...,13,16,17,17,18,22,24,26,0.236596,0.000000
4,"Blount, Alabama, US",Republican,Blount,AL,Alabama,33.990440,-86.562711,57840.0,28607.0,29406.0,...,12,13,14,16,17,18,20,20,0.304404,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3014,"Washakie, Wyoming, US",Republican,Washakie,WY,Wyoming,43.909060,-107.679282,7885.0,4075.0,3989.0,...,5,5,5,5,5,6,5,5,0.264583,0.000000
3015,"Weston, Wyoming, US",Republican,Weston,WY,Wyoming,43.843456,-104.556904,6967.0,3634.0,3293.0,...,0,0,0,0,0,0,0,0,0.000000,0.000000
3016,"Broomfield, Colorado, US",Democrat,Broomfield,CO,Colorado,39.963039,-105.058542,69267.0,33991.0,34350.0,...,68,81,81,88,88,92,95,98,0.381081,0.204545
3017,"Miami-Dade, Florida, US",Democrat,Miami-Dade,FL,Florida,25.607895,-80.587502,2761581.0,1336041.0,1415755.0,...,6487,7058,7459,7712,8066,8326,8824,9045,0.556344,0.320333


# Now let's explore the feature of when different amenities and orders were put in by governors

In [48]:
#First lets look at when gyms/entertainment centers were closed down
#Drop na values from the entertainment/gym column and convert to date time from ordinal value
#It should be noted that as na values are dropped, the amount of data will get smaller. However, we want data which contains all
# of the features we are interested in, therefore, this will be inevitable
joined_data = joined_data.dropna(axis=0,subset=['entertainment/gym'])
#Convert ordinal time to date time
joined_data['entertainment/gym'] = joined_data['entertainment/gym'].astype(int)
joined_data['entertainment/gym_closing_date'] = joined_data['entertainment/gym'].apply(lambda x: datetime.date.fromordinal(x))
joined_data['entertainment/gym_closing_date'] = pd.to_datetime(joined_data['entertainment/gym_closing_date'])

In [49]:
#Calculate how many days gyms have been closed as of 4/18/2020 
ending_date = pd.to_datetime('20200418',format='%Y%m%d')
joined_data['gym/entertainment_closed_duration'] = ending_date - joined_data['entertainment/gym_closing_date']

In [51]:
#Next lets look at when stay at home orders were issued
#Drop na values from the stay at home orders column and convert to date time from ordinal value
joined_data = joined_data.dropna(axis=0,subset=['stay at home'])
#Convert ordinal time to date time
joined_data['stay_at_home_issued_date'] = joined_data['stay at home'].astype(int)
joined_data['stay_at_home_issued_date'] = joined_data['stay_at_home_issued_date'].apply(lambda x: datetime.date.fromordinal(x))
joined_data['stay_at_home_issued_date'] = pd.to_datetime(joined_data['stay_at_home_issued_date'])

In [52]:
#Calculate how many days stay at home orders have been in place as of 4/18/2020 
ending_date = pd.to_datetime('20200418',format='%Y%m%d')
joined_data['stay_at_home_duration'] = ending_date - joined_data['stay_at_home_issued_date']

In [54]:
joined_data

,Combined_Key,Political_Party,CountyName,StateName,State,lat,lon,PopulationEstimate2018,PopTotalMale2017,PopTotalFemale2017,...,4/15/20,4/16/20,4/17/20,4/18/20,average_dtd_growth,avg_death_rate,entertainment/gym_closing_date,gym/entertainment_closed_duration,stay_at_home_issued_date,stay_at_home_duration
0,"Autauga, Alabama, US",Republican,Autauga,AL,Alabama,32.540091,-86.645649,55601.0,27007.0,28497.0,...,24,26,26,25,0.330657,0.125000,2020-03-28,21 days,2020-04-04,14 days
1,"Baldwin, Alabama, US",Republican,Baldwin,AL,Alabama,30.738314,-87.726272,218022.0,103225.0,109403.0,...,91,101,103,109,0.439499,0.227273,2020-03-28,21 days,2020-04-04,14 days
2,"Barbour, Alabama, US",Democrat,Barbour,AL,Alabama,31.874030,-85.397327,24881.0,13335.0,11935.0,...,12,14,15,18,0.200266,0.000000,2020-03-28,21 days,2020-04-04,14 days
3,"Bibb, Alabama, US",Republican,Bibb,AL,Alabama,32.999024,-87.125260,22400.0,12138.0,10530.0,...,18,22,24,26,0.236596,0.000000,2020-03-28,21 days,2020-04-04,14 days
4,"Blount, Alabama, US",Republican,Blount,AL,Alabama,33.990440,-86.562711,57840.0,28607.0,29406.0,...,17,18,20,20,0.304404,0.000000,2020-03-28,21 days,2020-04-04,14 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3013,"Uinta, Wyoming, US",Republican,Uinta,WY,Wyoming,41.289323,-110.553036,20299.0,10335.0,10160.0,...,4,4,6,6,0.196970,0.000000,2020-03-19,30 days,2020-03-28,21 days
3014,"Washakie, Wyoming, US",Republican,Washakie,WY,Wyoming,43.909060,-107.679282,7885.0,4075.0,3989.0,...,5,6,5,5,0.264583,0.000000,2020-03-19,30 days,2020-03-28,21 days
3015,"Weston, Wyoming, US",Republican,Weston,WY,Wyoming,43.843456,-104.556904,6967.0,3634.0,3293.0,...,0,0,0,0,0.000000,0.000000,2020-03-19,30 days,2020-03-28,21 days
3016,"Broomfield, Colorado, US",Democrat,Broomfield,CO,Colorado,39.963039,-105.058542,69267.0,33991.0,34350.0,...,88,92,95,98,0.381081,0.204545,2020-03-17,32 days,2020-03-26,23 days
